# Saliva Data Analysis

In [ ]:
from pathlib import Path
import json

import pandas as pd
import numpy as np
import pingouin as pg

import biopsykit as bp
from biopsykit.stats import StatsPipeline
from biopsykit.io import load_long_format_csv

from fau_colors import cmaps

import matplotlib.pyplot as plt
import seaborn as sns

from carwatch_analysis.utils import describe_groups_df
from carwatch_analysis.stats import create_unique_night_id, stats_pipeline_saliva_samples, stats_pipeline_saliva_features
from carwatch_analysis.plotting import boxplot_saliva_features

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)

theme_kwargs = {
    "context": "talk",
    "style": "ticks",
    "palette": palette
}
theme_kwargs_scale = {
    "context": "talk",
    "style": "ticks",
    "palette": palette,
    "font_scale": 1.25
}

tight_layout_params_boxplot = dict(rect=(0, 0, 0.85, 1.0), pad=0.2)

sns.set_theme(**theme_kwargs)

plt.rcParams["figure.figsize"] = (10,5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

export = True

pg.options["round"] = 4

palette

In [ ]:
base_path = Path("../..")
export_path = base_path.joinpath("exports")
result_path = base_path.joinpath("results")
plot_path = result_path.joinpath("plots")
stats_path = result_path.joinpath("statistics")

bp.utils.file_handling.mkdirs([result_path, plot_path, stats_path])

## Import

### Cortisol Samples

In [ ]:
cort_samples = load_long_format_csv(export_path.joinpath("cortisol_samples_cleaned.csv")).reset_index("cortisol")
cort_samples = create_unique_night_id(cort_samples)

cort_samples.head()

### Cortsol Features

In [ ]:
cort_features = load_long_format_csv(export_path.joinpath("cortisol_features_cleaned.csv"))
cort_features = create_unique_night_id(cort_features)

cort_features.head()

In [ ]:
car = bp.protocols.CAR()
car.add_saliva_data(cort_samples, saliva_type="cortisol", sample_times=[0, 15, 30, 45, 60])

## General Config

In [ ]:
condition_order = ["Spontaneous", "Known Alarm", "Unknown Alarm"]

## Plots and Statistics

### Weekend vs. Weekday

#### Class Distribution

In [ ]:
describe_groups_df(cort_samples, "weekend")

In [ ]:
df_stacked = bp.utils.dataframe_handling.stack_groups_percent(
    cort_samples, 
    hue="condition", 
    stacked="weekend", 
    order=condition_order
)

display(df_stacked.T)

In [ ]:
fig, ax = plt.subplots()
bp.plotting.stacked_barchart(data=df_stacked, order=condition_order, ylabel="Recorded Nights [%]", ax=ax)
fig.tight_layout()

if export:
    fig.savefig(plot_path.joinpath("img_barchart_weekdays.pdf"), transparent=True)

#### Raw Cortisol

##### Statistics

In [ ]:
variable = "weekend"

pipeline = stats_pipeline_saliva_samples(cort_samples, variable=variable)
pipeline.export_statistics(stats_path.joinpath(f"stats_cortisol_raw_{variable}.xlsx"))
pipeline.display_results(posthoc=False)

##### Plots

In [ ]:
hue = "weekend"

fig, ax = plt.subplots()

car.car_saliva_plot("cortisol", hue=hue, ax=ax)

if export:
    fig.savefig(plot_path.joinpath(f"img_car_{hue}.pdf"), transparent=True)

#### Features

##### Statistics

In [ ]:
variable = "weekend"

pipeline = stats_pipeline_saliva_features(cort_features, variable=variable)
pipeline.export_statistics(stats_path.joinpath(f"stats_cortisol_features_{variable}.xlsx"))
pipeline.display_results(posthoc=True)

##### Plots

In [ ]:
boxplot_saliva_features(cort_features, "weekend", pipeline, export, plot_path)

### Chronotype

#### Class Distribution

In [ ]:
describe_groups_df(cort_samples, "chronotype")

#### Raw Cortisol

##### Statistics

In [ ]:
variable = "chronotype"

pipeline = stats_pipeline_saliva_samples(cort_samples, variable=variable)
pipeline.export_statistics(stats_path.joinpath(f"stats_cortisol_raw_{variable}.xlsx"))
pipeline.display_results(posthoc=False)

##### Plots

In [ ]:
hue = "chronotype"
hue_order = ["Morning", "Intermediate", "Evening"]

fig, ax = plt.subplots()

car.car_saliva_plot("cortisol", hue=hue, hue_order=hue_order, ax=ax)

fig.tight_layout()
if export:
    fig.savefig(plot_path.joinpath(f"img_car_{hue}.pdf"), transparent=True)

#### Features

##### Statistics

In [ ]:
variable = "chronotype"

pipeline = stats_pipeline_saliva_features(cort_features, variable=variable, equal_var=True)
pipeline.export_statistics(stats_path.joinpath(f"stats_cortisol_features_{variable}.xlsx"))
pipeline.display_results(posthoc=False)

##### Plots

In [ ]:
boxplot_saliva_features(cort_features, "chronotype", pipeline, export, plot_path)

### MEQ

#### Linear Regression

In [ ]:
reg_results = cort_features.reset_index().groupby(["saliva_feature", "condition"]).apply(
    lambda df: pg.linear_regression(df["MEQ"], df["cortisol"], remove_na=True)
)
reg_results = reg_results.droplevel(-1).set_index("names", append=True)
reg_results = reg_results.drop("Intercept", level="names").droplevel(-1)
reg_results

#### Plots

In [ ]:
data_regplot = cort_features.xs("auc_i", level="saliva_feature")

fig, ax = plt.subplots()
data_regplot.groupby("condition").apply(
    lambda df: sns.regplot(data=df.reset_index(), x="MEQ", y="cortisol", ax=ax, label=df.name)
)
ax.legend(loc="upper right", fontsize="smaller")
fig.tight_layout()

### Within Ideal Bedtime

#### Class Distribution

In [ ]:
describe_groups_df(cort_samples, "within_ideal_bed_time")

#### Raw Cortisol

##### Statistics

In [ ]:
variable = "within_ideal_bed_time"

pipeline = stats_pipeline_saliva_samples(cort_samples, variable=variable)
pipeline.export_statistics(stats_path.joinpath(f"stats_cortisol_raw_{variable}.xlsx"))
pipeline.display_results(posthoc=False)

##### Plots

In [ ]:
hue = "within_ideal_bed_time"
hue_order = [True, False]

fig, ax = plt.subplots()

car.car_saliva_plot("cortisol", hue=hue, hue_order=hue_order, ax=ax)

fig.tight_layout()
if export:
    fig.savefig(plot_path.joinpath(f"img_car_{hue}.pdf"), transparent=True)

#### Features

##### Statistics

In [ ]:
variable = "within_ideal_bed_time"

pipeline = stats_pipeline_saliva_features(cort_features, variable=variable)
pipeline.export_statistics(stats_path.joinpath(f"stats_cortisol_features_{variable}.xlsx"))
pipeline.display_results(posthoc=True)

##### Plots

In [ ]:
boxplot_saliva_features(cort_features, "within_ideal_bed_time", pipeline, export, plot_path)

### Wakeup Source

#### Raw Cortisol

##### Statistics

In [ ]:
variable = "wakeup_source"

pipeline = stats_pipeline_saliva_samples(cort_samples, variable=variable)
pipeline.export_statistics(stats_path.joinpath(f"stats_cortisol_raw_{variable}.xlsx"))
pipeline.display_results(posthoc=False)

##### Plots

In [ ]:
hue = "wakeup_source"
hue_order = ["Spontaneous", "Alarm", "Not Specified"]

fig, ax = plt.subplots()

car.car_saliva_plot("cortisol", hue=hue, hue_order=hue_order, ax=ax)

fig.tight_layout()
if export:
    fig.savefig(plot_path.joinpath(f"img_car_{hue}.pdf"), transparent=True)

#### Features

##### Statistics

In [ ]:
variable = "wakeup_source"

pipeline = stats_pipeline_saliva_features(cort_features, variable=variable)
pipeline.export_statistics(stats_path.joinpath(f"stats_cortisol_features_{variable}.xlsx"))
pipeline.display_results(posthoc=False)

##### Plots

In [ ]:
boxplot_saliva_features(cort_features, "wakeup_source", pipeline, export, plot_path)

### Condition

#### Class Distribution

In [ ]:
describe_groups_df(cort_samples, "condition")

#### Raw Cortisol

##### Statistics

In [ ]:
variable = "condition"

pipeline = stats_pipeline_saliva_samples(cort_samples, variable=variable)
pipeline.export_statistics(stats_path.joinpath(f"stats_cortisol_raw_{variable}.xlsx"))
pipeline.display_results(posthoc=False)

##### Plots

In [ ]:
hue = "condition"
hue_order = ["Spontaneous", "Known Alarm", "Unknown Alarm"]

fig, ax = plt.subplots()

car.car_saliva_plot("cortisol", hue=hue, hue_order=hue_order, ax=ax)

fig.tight_layout()
if export:
    fig.savefig(plot_path.joinpath(f"img_car_{hue}.pdf"), transparent=True)

#### Features

##### Statistics

In [ ]:
variable = "condition"

pipeline = stats_pipeline_saliva_features(cort_features, variable=variable)
pipeline.export_statistics(stats_path.joinpath(f"stats_cortisol_features_{variable}.xlsx"))
pipeline.display_results(posthoc=False)

##### Plots

In [ ]:
boxplot_saliva_features(cort_features, "condition", pipeline, export, plot_path)

### Pairplots

In [ ]:
g = sns.pairplot(data=np.log(cort_samples["cortisol"].unstack("sample")).reset_index("condition"), hue="condition", corner=True)
g = g.map_lower(sns.kdeplot, levels=3, color=".2", alpha=0.5)

In [ ]:
g = sns.pairplot(data=cort_features["cortisol"].unstack("saliva_feature").reset_index("condition"), hue="condition", corner=True)
#g = g.map_lower(sns.kdeplot, levels=3, color=".2", alpha=0.5)